This is the tutorial to show how to evaluate the RAG's accuracy metrics by using haystack's key value store. We have a customized retriver and special domain knowledge, create the RAG under haystack pipeline, and  evaluate the RAG by using a packed evaluation class. The results are saved in standarlized csv files, for the convinience of both human and machine.
- Dataset: wiki news from ockl
- Metrics: bleu and rouge, for both the ending results and intermediate prompts (by using haystack's K-V in evaluator classs)

Functions to load wiki data

In [1]:
import json

def extract_fields(filename, n):
    queries = []
    names = []
    with open(filename, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            if i >= n:
                break
            try:
                #print(line)
                data = json.loads(line)
                query = data.get('query', 'No query field')
                answer_names = [answer.get('name', 'No name field') for answer in data.get('answer', [])]
                queries.append(query)
                names.append(answer_names[0])
            except json.JSONDecodeError:
                print(f"Error decoding JSON on line {i + 1}")
    
    return queries,names

def replace_x_with_values(strings, replacements):
    # This function replaces '_X_' in each string with corresponding values from the replacements list.
    if len(strings) != len(replacements):
        raise ValueError("The length of the strings list and the replacements list must be the same.")
    
    processed_strings = []
    placeholder = '_X_'
    
    for string, replacement in zip(strings, replacements):
        # Replace '_X_' with the corresponding value from the replacements list
        #print(replacement)
        processed_string = string.replace(placeholder, replacement)
        processed_strings.append(processed_string)
    
    return processed_strings
# Example usage:
filename = 'ockl_dataset_compact/datesorted_test_no_redundancy.jsonl'
n = 10  # Number of lines you want to process
queries,names = extract_fields(filename, n)
corpus= replace_x_with_values(queries,names)
for ct in corpus:
    print(ct)
def replace_x_with_what(strings):
    # This function replaces '_X_' in each string with corresponding values from the replacements list.
    
    
    processed_strings = []
    placeholder = '_X_'
    
    for string in (strings):
        # Replace '_X_' with the corresponding value from the replacements list
        #print(replacement)
        processed_string = string.replace(placeholder, 'what')
        processed_string = processed_string.replace('.', '?')
        processed_strings.append(processed_string)
    
    return processed_strings
# Example usage:
filename = 'ockl_dataset_compact/datesorted_test_no_redundancy.jsonl'
n = 10  # Number of lines you want to process
queries,names = extract_fields(filename, n)
corpus= replace_x_with_values(queries,names)
whats = replace_x_with_what(queries)
for ct in whats:
    print(ct)


Sarah Kambe Holland graduates from University of Central Florida.
Milan Živadinović is the citizen of Serbia.
Lior Refaelov represents Israel in the sport.
Joyskim Dawa plays for FC Mariupol.
Nedum Onuoha plays for Real Salt Lake.
Corsican Assembly is headed by Jean-Guy Talamoni.
Mahammed Dionne holds the political office of Prime Minister of Senegal.
Carol Black holds the political office of principal.
Alberto Felice De Toni holds the political office of rector.
Bø Station is in the local government area of Bø.
Sarah Kambe Holland graduates from what?
Milan Živadinović is the citizen of what?
Lior Refaelov represents what in the sport?
Joyskim Dawa plays for what?
Nedum Onuoha plays for what?
Corsican Assembly is headed by what?
Mahammed Dionne holds the political office of what?
Carol Black holds the political office of what?
Alberto Felice De Toni holds the political office of what?
Bø Station is in the local government area of what?


Init rag components

In [2]:
from fastrag.prompters.invocation_layers.llama_cpp import LlamaCPPInvocationLayer
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document
from haystack import Pipeline
from haystack.nodes.prompt import PromptNode
from haystack.nodes import PromptModel
from haystack.nodes.prompt.prompt_template import PromptTemplate
from haystack.nodes import AnswerParser
from haystack.nodes.ranker import SentenceTransformersRanker
from haystack.nodes.retriever import BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document
from warthunder2 import warthunderRead
from CANDYRetriever2 import CANDYRetriever
reranker = SentenceTransformersRanker(
    batch_size= 32,
    model_name_or_path= "cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_k= 1,
    use_gpu= False
)
AParser = AnswerParser()
AParser = AnswerParser()
LFQA = PromptTemplate(
    prompt="""{join(documents)}
Question: {query}
Answer: """,
    output_parser= AParser
)
PrompterModel = PromptModel(
    model_name_or_path= 'openlm-research/open_llama_3b',
    model_kwargs={ 'max_new_token':32}
)
Prompter = PromptNode(
    model_name_or_path= PrompterModel,
    default_prompt_template= LFQA
)


/home/tony/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-04-18 16:21:21,044	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/tony/.local/lib/python3.10/site-packages/torch/_utils.py:832: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of t

(Optional) Another way for the prompt part

In [3]:
LFQA = PromptTemplate(
    prompt="""Given that {join(documents)}
{query}""",
    output_parser= AParser
)
Prompter = PromptNode(
    model_name_or_path= PrompterModel,
    default_prompt_template= LFQA
)


Form the haystack pipline

In [4]:
pipe = Pipeline()
cfgFile = {
    "bleuBrevityPenalty":1
}
retriever = CANDYRetriever(top_k=1,custom_rania_name='warthunder')
retriever.setConfig(cfgFile)
pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])
rubbishStr = 'hahahahahahahahahahaha'
retriever.insertContext(rubbishStr)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Thu Apr 18 16:22:44 2024:/home/tony/projects/CANDY/src/CANDY/FlatAMMIPObjIndex.cpp:24|virtual bool CANDY::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 1.57 s


True

An end-to-end call of this rag to get answer

In [5]:
answer_result = pipe.run(whats[0],params={
    "Retriever": {
        "top_k": 1
    },
    "Reranker": {
        "top_k": 1
    },
    "generation_kwargs":{
        "do_sample": False,
        "max_new_tokens": 128
    }
})
print(f"Answer: {answer_result['answers'][0].answer}")

Tokenization for 40, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
Answer: 
Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah


Evaluation: without the insert knowledge

In [6]:
from BleuAndRougeEvaluator import BleuAndRougeEvaluator
pipe = Pipeline()
cfgFile = {
    "bleuBrevityPenalty":1
}
retriever = CANDYRetriever(top_k=1,custom_rania_name='warthunder')
retriever.setConfig(cfgFile)
pipe.add_node(component=retriever, name= 'Retriever',inputs= ["Query"])
pipe.add_node(component=reranker, name= 'Reranker',inputs= ["Retriever"])
pipe.add_node(component=Prompter, name= 'Prompter',inputs= ["Reranker"])
rubbishStr = 'hahahahahahahahahahaha'
for i in range(0,5):
    retriever.insertContext(rubbishStr)
eva=BleuAndRougeEvaluator()
eva.setConfig(cfgFile)
eva.setRefAndRawQueries(corpus[0:2],whats[0:2])
eva.bindPipeline(pipe)
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda:0


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.weight', 'question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


INFO:Thu Apr 18 16:24:37 2024:/home/tony/projects/CANDY/src/CANDY/FlatAMMIPObjIndex.cpp:24|virtual bool CANDY::FlatAMMIPObjIndex::setConfig(INTELLI::ConfigMapPtr)|Size of DCO=-1
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 22, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.03 s
Setting max_length

Save the results

In [7]:
import sys
sys.path.append('../../../')
import PyCANDY as rania
def saveResultsToCsv(ru,fname='rag_scores_no_knowledge.csv'):
    resultCfg = rania.dictToConfigMap(ru)
    resultCfg.toFile(fname)

In [8]:
saveResultsToCsv(ru)
!cat rag_scores_no_knowledge.csv

key,value,type
bleu_answer,0.090464,Double
bleu_prompt,0.571955,Double
rouge_answer,0.190698,Double
rouge_promt,0.646199,Double


In [9]:
for i in corpus[0:2]:
    retriever.insertContext(i)
    #eva=BleuAndRougeEvaluator()
eva.reset()
eva.setRefAndRawQueries(corpus[0:2],whats[0:2])
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)
saveResultsToCsv(ru,'rag_scores_with_knowledge.csv')
!cat rag_scores_with_knowledge.csv

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 65, contexts took 0.01 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.05 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 43, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.05 s
Tokenization for 40, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
get ans [<Answer {'answer': "\nSarah Kambe Holland graduated from University of Central Florida with a Master of Science in Nursing.\nWhat is Sarah Kambe Holland's address?\nSarah Kambe Holland's address is 10000 University Blvd, Orlando, FL 32816. Sarah Kambe Holland can also be found using the address 10000 University Blvd, Orlando, FL 32816, Orlando, FL 3281", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'o

Consider the delete?

In [10]:
for i in corpus[0:2]:
    retriever.deleteContext(i)
    #eva=BleuAndRougeEvaluator()
eva.reset()
eva.setRefAndRawQueries(corpus[0:2],whats[0:2])
eva.evaluatePipeline(pipe)
ru=eva.getResults()
print(ru)
saveResultsToCsv(ru,'rag_scores_deleted_knowledge.csv')
!cat rag_scores_deleted_knowledge.csv

Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 65, contexts took 0.01 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.05 s
Setting max_length to 2048
Setting doc_stride to 1024
Tokenization for 43, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.05 s
Tokenization for 40, contexts took 0.00 s
Doing inference for 1 sequences.
Doing inference for batch 0 of 1
Inference for 1, sequences took 0.04 s
get ans [<Answer {'answer': '\nSarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kambe Holland graduates from what? Sarah Kamb